In [3]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('diabetes.csv')

In [5]:
data.head()

,record_id,patient_id,measurement_date,measurement_27,measurement_28,measurement_37,measurement_42,measurement_43,measurement_68,measurement_76,measurement_89,measurement_65,measurement_119,measurement_67,measurement_107
0,0,69,2013-04-11,7.29,33.40,4.02,30.5,47.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,69,2013-04-30,7.16,24.57,4.08,39.7,54.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,69,2013-05-24,6.65,24.95,NaN,34.5,50.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,109,2013-02-06,6.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,109,2015-06-25,6.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
msd = pd.read_csv('measurements_dict.csv')

In [19]:
df = data[data['measurement_27'].isna() == False]

In [30]:
df.shape

(33522, 15)

In [29]:
for column in df.drop(['record_id', 'patient_id', 'measurement_date'], axis=1):
    print(str(df[column].isna().sum()) + " " + column)

0 measurement_27
15531 measurement_28
12130 measurement_37
12665 measurement_42
12613 measurement_43
32770 measurement_68
32959 measurement_76
30520 measurement_89
31025 measurement_65
32779 measurement_119
33415 measurement_67
33425 measurement_107


In [45]:
df['measurement_28'].notna().values.sum()

17991

In [44]:
np.logical_and(df['measurement_28'].notna().values, df['measurement_27'].notna().values).sum()

17991

In [47]:
print(np.logical_and(df['measurement_28'].notna().values, df['measurement_42'].notna().values).sum())
print(np.logical_and(df['measurement_28'].notna().values, df['measurement_43'].notna().values).sum())
print(np.logical_and(df['measurement_42'].notna().values, df['measurement_43'].notna().values).sum())

16868
16871
20764


In [54]:
temp1 = np.logical_and(df['measurement_28'].notna().values, df['measurement_37'].notna().values)
temp2 = np.logical_and(df['measurement_42'].notna().values, df['measurement_43'].notna().values)
np.logical_and(temp1, temp2).sum()

12130

In [58]:
df2 = df.drop(['measurement_68','measurement_76','measurement_89','measurement_65',
               'measurement_119','measurement_67','measurement_107'], axis=1)
df2 = df2.dropna()